WITH t1 AS
  (-- Информация о полётах и самолётах --
 SELECT s.aircraft_code,
        count(s.seat_no) count_seat,
        ac.model
   FROM dst_project.seats s
   JOIN dst_project.aircrafts ac ON ac.aircraft_code = s.aircraft_code
   GROUP BY s.aircraft_code,
            ac.model),
     t2 AS
  (-- Информация о полётах и стоимости проданных билетов (группировка)
 SELECT tf.flight_id,
        count(tf.ticket_no) count_ticket,
        sum(tf.amount) flight_profit
   FROM dst_project.ticket_flights tf
   GROUP BY tf.flight_id),
     t3 AS
  (-- Информация о полётах и кол-во проданных билетов на разные классы
 SELECT tf.flight_id,
        count(CASE
                  WHEN tf.fare_conditions = 'Business' THEN tf.fare_conditions
              END) business_ticket,
        count(CASE
                  WHEN tf.fare_conditions = 'Comfort' THEN tf.fare_conditions
              END) comfort_ticket,
        count(CASE
                  WHEN tf.fare_conditions = 'Economy' THEN tf.fare_conditions
              END) economy_ticket
   FROM dst_project.ticket_flights tf
   GROUP BY 1)
SELECT -- *,
 f.flight_id,
 f.arrival_airport,
 t1.model,
 t1.count_seat,
 t2.count_ticket*100/t1.count_seat perc_ticket_sold,
 t2.count_ticket,
 t3.business_ticket,
 t3.economy_ticket,
 t2.flight_profit,
 extract(epoch
         FROM (f.actual_arrival - f.actual_departure)) time_flying, -- продолжительность полёта, секунд
 date_part('month', f.scheduled_departure) month_flight
FROM dst_project.flights f
JOIN t1 ON f.aircraft_code = t1.aircraft_code
JOIN t2 ON f.flight_id = t2.flight_id
JOIN t3 ON f.flight_id = t3.flight_id
WHERE f.departure_airport = 'AAQ'
  AND (date_trunc('month', f.scheduled_departure) in ('2017-01-01',
                                                      '2017-02-01',
                                                      '2016-12-01'))
  AND status not in ('Cancelled')
ORDER BY flight_profit